<a href="https://colab.research.google.com/github/anilbhatt1/Deep_Learning_EVA4_Phase2/blob/master/E4P2S9_Simple_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Mon Nov 16 12:54:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    33W /  70W |    983MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Source : https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from torchtext import data

In [ ]:
SEED = 1234

torch.manual_seed(SEED)                      # We are uding seed to ensure that we get similar data while splitting train & test data
torch.backends.cudnn.deterministic = True
txt = data.Field(tokenize = 'spacy')
lbl = data.LabelField(dtype = torch.float)

#### train_data is organized in such a way that a single element contains a field, 'text', along with all the tokens representing the text, and a 'label' field that contains the label of the text (eg: in this case POS, NEG for sentiment analysis)

In [ ]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(txt, lbl)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 22.9MB/s]


In [ ]:
print('Length of train_data:',len(train_data), 'Type:', type(train_data))
print('Length of test_data :',len(test_data), 'Type:', type(test_data))
print(train_data.fields)
print(test_data.fields)
print(vars(train_data.examples[0]))   # vars -> Built-in function, with an argument equivalent to object.dict.
print(vars(test_data.examples[0])) 
print(vars(train_data[0]))

Length of train_data: 25000 Type: torchtext.datasets.imdb.IMDB
Length of test_data : 25000 Type: torchtext.datasets.imdb.IMDB
{'text': <torchtext.data.field.Field object at 0x7f6cecc7be80>, 'label': <torchtext.data.field.LabelField object at 0x7f6c96b010f0>}
{'text': <torchtext.data.field.Field object at 0x7f6cecc7be80>, 'label': <torchtext.data.field.LabelField object at 0x7f6c96b010f0>}
{'text': ['Upon', 'the', 'first', 'viewing', ',', 'I', 'found', 'this', 'tale', 'to', 'be', 'at', 'least', 'less', 'annoying', 'than', 'other', 'Cannon', 'Movie', 'Tales', '.', 'After', 'many', 'more', ',', 'I', 'think', 'it', "'s", 'one', 'of', 'the', 'best', '.', 'Some', 'of', 'the', 'songs', 'are', 'pretty', 'bad', ',', 'especially', 'the', 'love', 'song', ',', 'but', 'two', 'things', 'stand', 'out', 'that', 'make', 'the', 'movie', ',', 'even', 'the', 'singing', ',', 'worthwhile', '.', 'One', 'is', 'the', 'art', 'direction', '.', 'Like', 'the', 'other', 'Cannon', 'Movie', 'Tales', ',', 'this', 'is'

##### Split the train_data further into train_data & valid_data

In [ ]:
import random
train_data, valid_data = train_data.split(split_ratio=0.8,random_state=random.seed(SEED))

In [ ]:
print('Length of train_data:',len(train_data))
print('Length of valid_data:',len(valid_data))
print('Length of test_data:',len(test_data))

Length of train_data: 20000
Length of valid_data: 5000
Length of test_data: 25000


### Building vocabulary. We take only top 25,000 words. Rest will be replaced with 'unk'.

In [ ]:
max_vocab_size = 25000
txt.build_vocab(train_data, max_size=max_vocab_size)
lbl.build_vocab(train_data)

In [ ]:
print('Unique words in txt:',len(txt.vocab))   # 25,002 words because of unk & pad. Pad is used to make batch_size same for shorter sentences
print('Unique words in lbl:',len(lbl.vocab))

Unique words in txt: 25002
Unique words in lbl: 2


In [ ]:
print(txt.vocab.freqs.most_common(25))  # Printing most commonly occuring 25 words

[('the', 231848), (',', 221111), ('.', 189822), ('a', 125351), ('and', 125008), ('of', 115525), ('to', 107014), ('is', 87145), ('in', 70266), ('I', 62048), ('it', 61433), ('that', 56207), ('"', 51222), ("'s", 49781), ('this', 48566), ('-', 42515), ('/><br', 40723), ('was', 39950), ('as', 34947), ('with', 34300), ('movie', 33959), ('for', 33733), ('film', 30956), ('The', 30116), ('but', 28019)]


## How to see vocabulary directly ? 
We can  see the vocabulary directly using either the stoi (string to int) or itos (int to string) method.

In [ ]:
print(txt.vocab.itos[:10])     # First 10
print(txt.vocab.itos[77:90])   # In between
print(txt.vocab.itos[-10:])    # Last 10

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']
['really', 'were', 'see', 'their', 'even', 'no', 'my', 'me', 'did', 'does', 'than', '...', 'much']
['flicked', 'floor.<br', 'floored', 'flounder', 'fluidity', 'fluids', 'flurry', 'focal', 'foibles', 'folded']


In [ ]:
print(txt.vocab.stoi)
print(lbl.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f6cecd27510>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, '.': 4, 'a': 5, 'and': 6, 'of': 7, 'to': 8, 'is': 9, 'in': 10, 'I': 11, 'it': 12, 'that': 13, '"': 14, "'s": 15, 'this': 16, '-': 17, '/><br': 18, 'was': 19, 'as': 20, 'with': 21, 'movie': 22, 'for': 23, 'film': 24, 'The': 25, 'but': 26, "n't": 27, '(': 28, 'on': 29, ')': 30, 'you': 31, 'are': 32, 'not': 33, 'have': 34, 'his': 35, 'be': 36, 'he': 37, 'one': 38, '!': 39, 'by': 40, 'at': 41, 'all': 42, 'an': 43, 'who': 44, 'from': 45, 'like': 46, 'they': 47, 'so': 48, 'her': 49, 'or': 50, 'about': 51, 'has': 52, "'": 53, 'out': 54, 'It': 55, 'just': 56, 'do': 57, '?': 58, 'some': 59, 'good': 60, 'more': 61, 'very': 62, 'would': 63, 'up': 64, 'what': 65, 'This': 66, 'there': 67, 'time': 68, 'can': 69, 'when': 70, 'which': 71, 'if': 72, 'she': 73, 'story': 74, 'had': 75, 'only': 76, 'really': 77, 'were': 78, 'see': 79, 'their': 80, 'even': 81, 'no': 82, 'my': 83, 'me': 84, 'did': 85, 'do

#### Creating iterator. Using buckeiterator that will return batch of examples where each example is of similar length, minimizing the amount of padding per example

In [ ]:
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                                            batch_size = batch_size,
                                                                            device = device)

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
     def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
         super().__init__()
         self.embedding = nn.Embedding(input_dim, embedding_dim)  
         self.rnn = nn.RNN(embedding_dim, hidden_dim)
         self.fc = nn.Linear(hidden_dim, output_dim)
     def forward(self, text):
         #text = [sentence len, batch size] --> eg: [900, 64] means 64 sentences with 900 words each. Length of sentence will vary for each batch.
         embedded = self.embedding(text)
         #embedded = [sentence len, batch size, emb dim] --> [900, 64, 100] adding one more dimension for embed dimension
         output, hidden = self.rnn(embedded)
         #output = [sentence len, batch size, hid dim] ---> [900, 64, 256]
         #hidden = [1, batch size, hid dim]            ---> [1, 64, 256]
         assert torch.equal(output[-1,:,:], hidden.squeeze(0))
         return self.fc(hidden.squeeze(0))    # squeeze is used to remove dimension 1, hidden.squeeze(0) -> [64,256], self.fc() -> [64,1]

In [ ]:
input_dim = len(txt.vocab)   # 25002
embedding_dim = 100
hidden_dim    = 256
output_dim    = 1            # only 2 values - pos or neg as this is sentiment analysis

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [ ]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc  = 0

    model.train()

    for idx, batch in enumerate(iterator):       
        optimizer.zero_grad()        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc  = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc  += acc.item()    

    return epoch_loss / len(iterator), epoch_acc / len(iterator)   

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc  = 0

    model.eval()

    with torch.no_grad():
        for idx, batch in enumerate(iterator):          
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc  = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc  += acc.item() 

    return epoch_loss / len(iterator), epoch_acc / len(iterator)        

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
n_epochs = 5
best_valid_loss = float('inf')

for epoch in range(n_epochs):
    
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/gdrive/My Drive/EVA4P2_S9/E4P2_S9_Simple_Senti_Analysis.pt')
       
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



Epoch: 01 | Epoch Time: 0m 16s
	Train Loss: 0.695 | Train Acc: 50.12%
	 Val. Loss: 0.696 |  Val. Acc: 50.55%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.693 | Train Acc: 49.94%
	 Val. Loss: 0.696 |  Val. Acc: 50.93%
Epoch: 03 | Epoch Time: 0m 16s
	Train Loss: 0.693 | Train Acc: 49.88%
	 Val. Loss: 0.696 |  Val. Acc: 50.57%
Epoch: 04 | Epoch Time: 0m 16s
	Train Loss: 0.693 | Train Acc: 50.10%
	 Val. Loss: 0.697 |  Val. Acc: 48.28%
Epoch: 05 | Epoch Time: 0m 16s
	Train Loss: 0.693 | Train Acc: 49.76%
	 Val. Loss: 0.697 |  Val. Acc: 48.24%


In [ ]:
len(txt.vocab)

25002